In [1]:
import numpy as np
from learning import load_dataset
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

Using TensorFlow backend.


In [2]:
z, u, F = load_dataset("../datasets/dynamics.npz")
dt = 0.02
N = len(z)

[learning.py] loading dataset (../datasets/dynamics.npz)
[learning.py] dataset loaded (151407 samples)


In [3]:
z.shape, u.shape, F.shape

((151407, 11), (151407, 3), (151407, 11))

In [4]:
df = pd.DataFrame({"x": z[:, 0], "y": z[:, 1], "theta": z[:, 2], "v_x": z[:, 3], "v_y": z[:, 4], "omega": z[:, 5], "phi": z[:, 6], 
                   "w_fl": z[:, 7], "w_fr": z[:, 8], "w_rl": z[:, 9], "w_rr": z[:, 10],
                   "u_s": u[:, 0], "u_t": u[:, 1], "u_b": u[:, 2]})
df.head()

,x,y,theta,v_x,v_y,omega,phi,w_fl,w_fr,w_rl,w_rr,u_s,u_t,u_b
0,219.004456,-54.789326,1.466342,-8.509562e-07,8.920291e-08,-0.000413,2.980232e-08,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,219.004456,-54.789326,1.466342,-8.509562e-07,8.920291e-08,-0.000413,2.980232e-08,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,219.004456,-54.789326,1.466352,-2.119278e-05,2.221578e-06,0.000542,7.450581e-08,0.000251,-0.000251,0.000251,-0.000251,0.0,0.0,0.0
3,219.004456,-54.789326,1.466348,2.657793e-05,-2.786238e-06,-0.000222,-1.490116e-07,-0.000154,0.000154,-0.000154,0.000154,0.0,0.0,0.0
4,219.004456,-54.789326,1.466350,-9.011736e-06,9.447957e-07,0.000101,7.450581e-09,0.000027,-0.000027,0.000027,-0.000027,0.0,0.0,0.0


In [5]:
labels = pd.DataFrame()
labels["x''"] = (F[:, 3] - z[:, 3])/dt
labels["y''"] = (F[:, 4] - z[:, 4])/dt
labels["theta''"] = (F[:, 5] - z[:, 5])/dt

In [23]:
Y = labels["x''"]
X = df
X = sm.add_constant(X)

def vif(X):
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif["features"] = X.columns

    return vif.round(1)

def fit(X, Y):
    reg = sm.OLS(Y, X).fit()
    return reg.summary()

fit(X, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    x''   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     603.2
Date:                Thu, 07 May 2020   Prob (F-statistic):               0.00
Time:                        02:32:07   Log-Likelihood:            -8.9852e+05
No. Observations:              151407   AIC:                         1.797e+06
Df Residuals:                  151392   BIC:                         1.797e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.0814      0.874     11.538      0.000       8.369      11.794
x             -0.1330      0.002    -75.455      0.000      -0.136      -0.130
y             -0.0045      0.002     -2.040      0.041      -0.009      -0.000
theta         -1.1603      0.173     -6.705      0.000      -1.499      -0.821
v_x           -0.0407      0.005     -8.795      0.000      -0.050      -0.032
v_y           -0.1925      0.007    -27.675      0.000      -0.206      -0.179
omega          0.0553      0.264      0.209      0.834      -0.462       0.573
phi           15.6111      2.550      6.121      0.000      10.612      20.610
w_fl           0.4652      0.113      4.117      0.000       0.244       0.687
w_fr          -0.6388      0.113     -5.647      0.000      -0.861      -0.417
w_rl          -0.1982      0.079     -2.500      0.012      -0.354      -0.043
w_rr           0.3714      0.080      4.663      0.000       0.215       0.528
u_s            4.5495      0.963      4.726      0.000       2.663       6.436
u_t           -2.4771      0.518     -4.779      0.000      -3.493      -1.461
u_b          -12.4433      1.328     -9.368      0.000     -15.047      -9.840
==============================================================================
Omnibus:                      878.839   Durbin-Watson:                   0.041
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1068.381
Skew:                          -0.118   Prob(JB):                    1.01e-232
Kurtosis:                       3.337   Cond. No.                     3.02e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
vif(X)

,VIF Factor,features
0,13.8,const
1,1.1,x
2,1.1,y
3,1.8,theta
4,1.1,v_x
5,1.8,v_y
6,6.9,omega
7,6.8,phi
8,707.9,w_fl
9,708.5,w_fr


In [42]:
X2 = X.drop(["w_fl", "w_rr", "w_rl", "w_fr", "omega", "phi", "u_b", "u_t"], axis="columns")

fit(X2, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    x''   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     1340.
Date:                Thu, 07 May 2020   Prob (F-statistic):               0.00
Time:                        11:42:23   Log-Likelihood:            -8.9871e+05
No. Observations:              151407   AIC:                         1.797e+06
Df Residuals:                  151400   BIC:                         1.798e+06
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.5231      0.589     17.870      0.000       9.369      11.677
x             -0.1328      0.002    -76.343      0.000      -0.136      -0.129
y             -0.0052      0.002     -2.384      0.017      -0.010      -0.001
theta         -1.2266      0.171     -7.173      0.000      -1.562      -0.891
v_x           -0.0399      0.005     -8.640      0.000      -0.049      -0.031
v_y           -0.1922      0.007    -27.799      0.000      -0.206      -0.179
u_s           -1.8682      0.439     -4.259      0.000      -2.728      -1.008
==============================================================================
Omnibus:                      776.446   Durbin-Watson:                   0.041
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              958.681
Skew:                          -0.102   Prob(JB):                    6.68e-209
Kurtosis:                       3.333   Cond. No.                         386.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
vif(X2)

,VIF Factor,features
0,1.2,x
1,1.1,y
2,4.1,theta
3,1.1,v_x
4,1.5,v_y
5,3.5,w_fr
6,1.0,u_s


In [43]:
X2["a_cos_theta"] = df["u_t"] * np.cos(df["theta"])
X2["b_cos_theta"] = df["u_b"] * np.cos(df["theta"])
X2["a_sin_theta"] = df["u_t"] * np.sin(df["theta"])
X2["b_sin_theta"] = df["u_b"] * np.sin(df["theta"])
# X2["delta_2"] = (df["u_t"] - df["theta"])**2
X2["v_sin_phi"] = np.sqrt(df["v_x"]**2 + df["v_y"]**2) * np.sin(df["u_s"])
X2 = X2.drop(["u_s", "theta", "y"], axis="columns")


fit(X2, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    x''   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     2424.
Date:                Thu, 07 May 2020   Prob (F-statistic):               0.00
Time:                        11:42:26   Log-Likelihood:            -8.9350e+05
No. Observations:              151407   AIC:                         1.787e+06
Df Residuals:                  151398   BIC:                         1.787e+06
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           6.2376      0.243     25.672      0.000       5.761       6.714
x              -0.1171      0.002    -69.571      0.000      -0.120      -0.114
v_x            -0.3393      0.006    -57.918      0.000      -0.351      -0.328
v_y            -0.2500      0.006    -39.133      0.000      -0.262      -0.237
a_cos_theta    52.5167      0.617     85.140      0.000      51.308      53.726
b_cos_theta   -91.9445      1.577    -58.304      0.000     -95.035     -88.854
a_sin_theta    15.1794      0.644     23.576      0.000      13.918      16.441
b_sin_theta    -7.5824      1.624     -4.670      0.000     -10.765      -4.400
v_sin_phi      -0.0458      0.008     -6.094      0.000      -0.061      -0.031
==============================================================================
Omnibus:                     1139.727   Durbin-Watson:                   0.042
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1635.579
Skew:                          -0.085   Prob(JB):                         0.00
Kurtosis:                       3.480   Cond. No.                     1.09e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.09e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
vif(X2)

,VIF Factor,features
0,1.1,const
1,1.1,x
2,1.8,v_x
3,1.7,v_y
4,1.8,a_cos_theta
5,1.0,b_cos_theta
6,1.6,a_sin_theta
7,1.0,b_sin_theta
8,1.0,v_sin_phi


In [ ]:
state = np.array([])
u = np.array([])
ext_state = np.array([1, state[0], state[3], state[4], u[1] * np.cos(state[2]), u[2] * np.cos(state[2]),
                     u[1] * np.sin(state[2]), u[2] * np.sin(state[2]),
                      np.sqrt(state[3]**2 + state[4]**2) * np.sin(u[0])
                     ])
x_coeffs = np.array([1.1, 1.1, 1.8, 1.7, 1.8, 1.0, 1.6, 1.0, 1.0])

In [36]:
fit(X2[["a_cos_theta", "b_cos_theta"]], Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    x''   R-squared (uncentered):                   0.049
Model:                            OLS   Adj. R-squared (uncentered):              0.049
Method:                 Least Squares   F-statistic:                              3904.
Date:                Thu, 07 May 2020   Prob (F-statistic):                        0.00
Time:                        02:35:32   Log-Likelihood:                     -8.9882e+05
No. Observations:              151407   AIC:                                  1.798e+06
Df Residuals:                  151405   BIC:                                  1.798e+06
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
a_cos_theta    31.0451      0.477     65.077      0.000      30.110      31.980
b_cos_theta  -106.4294      1.617    -65.824      0.000    -109.598    -103.260
==============================================================================
Omnibus:                      583.348   Durbin-Watson:                   0.038
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              802.173
Skew:                           0.006   Prob(JB):                    6.46e-175
Kurtosis:                       3.356   Cond. No.                         3.41
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
import matplotlib.pyplot as plt

# plt.scatter(X, Y, s=1)
# plt.plot(X, reg.intercept_ + reg.coef_[0] * X)
# plt.show()


In [56]:
Y = labels["y''"]
X = df.drop(["omega", "x", "w_rr", "w_fl", "w_fr"], axis="columns")
# X = sm.add_constant(X)

fit(X, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    y''   R-squared (uncentered):                   0.050
Model:                            OLS   Adj. R-squared (uncentered):              0.050
Method:                 Least Squares   F-statistic:                              884.5
Date:                Thu, 07 May 2020   Prob (F-statistic):                        0.00
Time:                        21:42:58   Log-Likelihood:                     -9.0630e+05
No. Observations:              151407   AIC:                                  1.813e+06
Df Residuals:                  151398   BIC:                                  1.813e+06
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
y             -0.1530      0.002    -66.537      0.000      -0.157      -0.148
theta         -5.9005      0.157    -37.562      0.000      -6.208      -5.593
v_x            0.1149      0.005     23.608      0.000       0.105       0.124
v_y           -0.1842      0.007    -27.022      0.000      -0.198      -0.171
phi          -12.9804      2.127     -6.103      0.000     -17.149      -8.812
w_rl           0.1123      0.004     28.253      0.000       0.105       0.120
u_s           -8.4080      0.952     -8.836      0.000     -10.273      -6.543
u_t            4.9072      0.513      9.573      0.000       3.903       5.912
u_b            6.3439      1.297      4.893      0.000       3.803       8.885
==============================================================================
Omnibus:                       58.572   Durbin-Watson:                   0.043
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               60.661
Skew:                          -0.034   Prob(JB):                     6.73e-14
Kurtosis:                       3.072   Cond. No.                     1.26e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.26e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [57]:
vif(X)

,VIF Factor,features
0,1.1,y
1,5.2,theta
2,1.1,v_x
3,1.6,v_y
4,4.3,phi
5,4.7,w_rl
6,4.3,u_s
7,1.9,u_t
8,1.1,u_b


In [61]:
X2 = X.drop(["phi", "w_rl"], axis="columns")

X2["a_cos_theta"] = df["u_t"] * np.cos(df["theta"])
X2["b_cos_theta"] = df["u_b"] * np.cos(df["theta"])
X2["a_sin_theta"] = df["u_t"] * np.sin(df["theta"])
X2["b_sin_theta"] = df["u_b"] * np.sin(df["theta"])
# X2["delta_2"] = (df["u_t"] - df["theta"])**2
X2["v_sin_phi"] = np.sqrt(df["v_x"]**2 + df["v_y"]**2) * np.sin(df["u_s"])

fit(X2, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    y''   R-squared (uncentered):                   0.107
Model:                            OLS   Adj. R-squared (uncentered):              0.107
Method:                 Least Squares   F-statistic:                              1514.
Date:                Thu, 07 May 2020   Prob (F-statistic):                        0.00
Time:                        21:44:18   Log-Likelihood:                     -9.0160e+05
No. Observations:              151407   AIC:                                  1.803e+06
Df Residuals:                  151395   BIC:                                  1.803e+06
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
y              -0.1383      0.002    -61.940      0.000      -0.143      -0.134
theta          -2.3550      0.097    -24.335      0.000      -2.545      -2.165
v_x             0.1694      0.006     27.117      0.000       0.157       0.182
v_y            -0.3919      0.007    -55.579      0.000      -0.406      -0.378
u_s            -9.5103      1.183     -8.042      0.000     -11.828      -7.193
u_t             7.1728      0.475     15.110      0.000       6.242       8.103
u_b             3.2603      1.243      2.622      0.009       0.823       5.697
a_cos_theta    -8.6824      0.650    -13.367      0.000      -9.956      -7.409
b_cos_theta    16.7642      1.680      9.980      0.000      13.472      20.056
a_sin_theta    55.0365      0.683     80.601      0.000      53.698      56.375
b_sin_theta  -108.7370      1.728    -62.929      0.000    -112.124    -105.350
v_sin_phi       0.1456      0.021      6.945      0.000       0.105       0.187
==============================================================================
Omnibus:                      254.426   Durbin-Watson:                   0.044
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              287.435
Skew:                           0.055   Prob(JB):                     3.84e-63
Kurtosis:                       3.182   Cond. No.                         839.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
vif(X2)

,VIF Factor,features
0,1.1,y
1,5.4,theta
2,1.9,v_x
3,2.1,v_y
4,4.9,w_rl
5,7.0,u_s
6,1.9,u_t
7,1.2,u_b
8,1.8,a_cos_theta
9,1.1,b_cos_theta


In [76]:
Y = labels["theta''"]
X = df.drop(["u_b", "v_x", "phi", "y", "w_rl", "w_fl", "w_fr", "w_rr"], axis="columns")
# X = sm.add_constant(X)

fit(X, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                theta''   R-squared (uncentered):                   0.125
Model:                            OLS   Adj. R-squared (uncentered):              0.125
Method:                 Least Squares   F-statistic:                              3598.
Date:                Thu, 07 May 2020   Prob (F-statistic):                        0.00
Time:                        21:54:31   Log-Likelihood:                     -6.1933e+05
No. Observations:              151407   AIC:                                  1.239e+06
Df Residuals:                  151401   BIC:                                  1.239e+06
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x             -0.0015      0.000     -5.682      0.000      -0.002      -0.001
theta          0.1177      0.015      8.012      0.000       0.089       0.147
v_y            0.0041      0.001      4.443      0.000       0.002       0.006
omega         -1.0450      0.016    -63.688      0.000      -1.077      -1.013
u_s          -10.3244      0.072   -143.924      0.000     -10.465     -10.184
u_t           -0.4028      0.073     -5.505      0.000      -0.546      -0.259
==============================================================================
Omnibus:                    24319.710   Durbin-Watson:                   0.772
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           459526.631
Skew:                          -0.125   Prob(JB):                         0.00
Kurtosis:                      11.531   Cond. No.                         296.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [68]:
vif(X)

,VIF Factor,features
0,1.1,x
1,2.0,theta
2,1.3,v_y
3,1.1,omega
4,1.1,u_s
5,1.7,u_t


In [77]:
X2 = X # .drop(["w_rr", "theta", "v_y"], axis="columns")

# X2["a_cos_theta"] = df["u_t"] * np.cos(df["theta"])
# X2["b_cos_theta"] = df["u_b"] * np.cos(df["theta"])
# X2["a_sin_theta"] = df["u_t"] * np.sin(df["theta"])
# X2["b_sin_theta"] = df["u_b"] * np.sin(df["theta"])
# X2["delta_2"] = (df["u_t"] - df["theta"])**2
X2["v_sin_phi"] = np.sqrt(df["v_x"]**2 + df["v_y"]**2) * np.sin(df["u_s"])

fit(X2, Y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                theta''   R-squared (uncentered):                   0.129
Model:                            OLS   Adj. R-squared (uncentered):              0.129
Method:                 Least Squares   F-statistic:                              3195.
Date:                Thu, 07 May 2020   Prob (F-statistic):                        0.00
Time:                        21:54:35   Log-Likelihood:                     -6.1898e+05
No. Observations:              151407   AIC:                                  1.238e+06
Df Residuals:                  151400   BIC:                                  1.238e+06
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x             -0.0015      0.000     -5.424      0.000      -0.002      -0.001
theta          0.1194      0.015      8.147      0.000       0.091       0.148
v_y            0.0043      0.001      4.662      0.000       0.002       0.006
omega         -1.1130      0.017    -67.141      0.000      -1.145      -1.081
u_s           -5.9254      0.183    -32.367      0.000      -6.284      -5.567
u_t           -0.4191      0.073     -5.740      0.000      -0.562      -0.276
v_sin_phi     -0.0857      0.003    -26.107      0.000      -0.092      -0.079
==============================================================================
Omnibus:                    23834.228   Durbin-Watson:                   0.773
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           429834.174
Skew:                          -0.124   Prob(JB):                         0.00
Kurtosis:                      11.251   Cond. No.                         732.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
vif(X2)

,VIF Factor,features
0,3.7,const
1,1.9,v_x
2,1.5,omega
3,10.9,phi
4,4.8,u_s
5,1.1,u_t
6,1.0,u_b
7,1.8,a_cos_theta
8,1.1,delta_2
9,7.4,v_sin_phi
